## Virtual Environment for Individual-Based Modeling

This virtual environment is based on an Agent-Based Modeling system using some assumptions that are derived from previousobservations of waterbirds’ behaviors within some habitats in the tropics.

Created on March 31, 2019 by Ralph Florent <r.florent@jacobs-university.de>

In [1]:
# Virtual Environment for Individual-Based Modeling (IBM)
#
# Created on March 31, 2019
#
# Authors:
#   Ralph Florent <r.florent@jacobs-university.de>
#   Davi Tavares <davi.tavares@leibniz-zmt.de>
#   Agostino Merico <a.merico@jacobs-university.de>
#
# Main entry point for the application

# ==============================================================================
# START: Preamble
# ==============================================================================

# -*- coding: utf-8 -*-
import config
import constants as CONST
from core import initialize, observe, update
from helpers import plot_figure, make_gif

# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# reload external python modules
%load_ext autoreload
%autoreload 2
%reload_ext autoreload


# ==============================================================================
# END: Preamble
# ==============================================================================

# ==============================================================================
# START: Application
# ==============================================================================

# main entry point for the application
# TODO: proper docs
def application():
    
    # pre-conditions
    config.init() # initialize internal config for the app
    time = 0 # define stopwatch for the process
    
    habitats, agents = initialize()
    observe(habitats, agents, time)
    
    # process for t times
    print('=> START: Processing random movements based on prob dist')
    
    for time in range(1, CONST.PROCESSING_TIME):
        agents = update(habitats, agents) # override agents when being updated
        observe(habitats, agents, time)
    
    print('=> END: Processing random movements based on prob dist')
    
    # post-conditions
    make_gif(CONST.MAIN_DIRECTORY, 'image.gif', CONST.IMAGE_STORAGE)
    plot_figure()

    
# run application
application()

# ==============================================================================
# END: Application
# ==============================================================================

=> START: Processing random movements based on prob dist
=> process update: 2
=> process update: 3
=> process update: 4
=> process update: 5
=> process update: 6
=> process update: 7
=> process update: 8
=> process update: 9
=> process update: 10
=> process update: 11
=> process update: 12
=> process update: 13
=> process update: 14
=> process update: 15
=> process update: 16
=> process update: 17
=> process update: 18
=> process update: 19
=> process update: 20
=> process update: 21
=> process update: 22
=> process update: 23
=> process update: 24
=> process update: 25
=> process update: 26
=> process update: 27
=> process update: 28
=> process update: 29
=> process update: 30
=> process update: 31
=> process update: 32
=> process update: 33
=> process update: 34
=> process update: 35
=> process update: 36
=> process update: 37
=> process update: 38
=> process update: 39
=> process update: 40
=> process update: 41
=> process update: 42
=> process update: 43
=> process update: 44
=> pr